In [1]:
!pip install folium pandas jupyter


  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached setuptools-76.1.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cach

In [2]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon


In [7]:
#Downloading the dataset

import pandas as pd
import requests
from io import BytesIO

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Descargar el archivo sin verificación SSL
response = requests.get(url, verify=False)
spacex_df = df  # Asignar el dataset a la variable que usarás más adelante


# Leer el contenido CSV desde la respuesta
df = pd.read_csv(BytesIO(response.content))

# Mostrar las primeras filas
df.head()


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [8]:
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df = launch_sites_df.drop_duplicates()
launch_sites_df.head()


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
26,VAFB SLC-4E,34.632834,-120.610745
36,KSC LC-39A,28.573255,-80.646895
49,CCAFS SLC-40,28.563197,-80.576820


In [9]:
import folium

# Coordenadas iniciales (NASA Johnson Space Center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Crear un mapa con folium
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Mostrar el mapa
site_map


In [11]:
from folium import DivIcon

# Crear un mapa centrado en NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Iterar sobre cada sitio de lanzamiento y agregar un círculo y un marcador
for index, row in launch_sites_df.iterrows():
    # Crear un círculo con radio de 1000 metros
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',  # Color naranja
        fill=True,
        fill_color='#d35400',
        popup=row['Launch Site']
    ).add_to(site_map)

    # Agregar un marcador con un texto encima
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Mostrar el mapa
site_map


In [12]:
# Agregar una columna 'marker_color' según el éxito o fracaso del lanzamiento
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [13]:
from folium.plugins import MarkerCluster

# Crear un objeto MarkerCluster
marker_cluster = MarkerCluster()

# Iterar sobre cada lanzamiento y agregarlo al MarkerCluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Agregar el cluster al mapa
site_map.add_child(marker_cluster)

# Mostrar el mapa
site_map


In [14]:
from folium.plugins import MousePosition

# Función para mostrar coordenadas en la esquina superior derecha
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Agregar al mapa
site_map.add_child(mouse_position)
site_map


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Radio aproximado de la Tierra en km
    R = 6373.0  

    # Convertir coordenadas a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Diferencias de coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distancia en km
    distance = R * c
    return distance


In [16]:
# Coordenadas de un sitio de lanzamiento (ejemplo: CCAFS LC-40)
launch_site_lat = 28.56319718
launch_site_lon = -80.57682003

# Coordenadas del punto en la costa más cercana (obtenidas con MousePosition)
coastline_lat = 28.56367  
coastline_lon = -80.57163  

# Calcular la distancia
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Crear y agregar un marcador con la distancia al mapa
from folium import Marker, DivIcon

distance_marker = Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline)
    )
)
site_map.add_child(distance_marker)


In [17]:
from folium import PolyLine

# Crear línea entre el sitio de lanzamiento y la costa
lines = PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (coastline_lat, coastline_lon)], 
    weight=2, 
    color='blue'
)

site_map.add_child(lines)
site_map


In [18]:
# Coordenadas de un sitio de lanzamiento (ejemplo: CCAFS LC-40)
launch_site_lat = 28.56319718
launch_site_lon = -80.57682003

# Coordenadas del punto de interés más cercano (ciudad, ferrocarril, carretera)
city_lat, city_lon = 28.3922, -80.6077   # Ciudad más cercana (ejemplo)
railway_lat, railway_lon = 28.5721, -80.5856  # Ferrocarril cercano
highway_lat, highway_lon = 28.5631, -80.5700  # Carretera cercana


In [19]:
# Calcular distancias
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

print(f"Distancia a la ciudad más cercana: {distance_city:.2f} km")
print(f"Distancia al ferrocarril más cercano: {distance_railway:.2f} km")
print(f"Distancia a la carretera más cercana: {distance_highway:.2f} km")


Distancia a la ciudad más cercana: 19.26 km
Distancia al ferrocarril más cercano: 1.31 km
Distancia a la carretera más cercana: 0.67 km


In [20]:
from folium import Marker, DivIcon

# Crear marcadores con la distancia a cada punto de interés
city_marker = Marker(
    [city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_city:.2f} KM</b></div>'
    )
)

railway_marker = Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#1e90ff;"><b>{distance_railway:.2f} KM</b></div>'
    )
)

highway_marker = Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#228B22;"><b>{distance_highway:.2f} KM</b></div>'
    )
)

# Agregar los marcadores al mapa
site_map.add_child(city_marker)
site_map.add_child(railway_marker)
site_map.add_child(highway_marker)


In [21]:
from folium import PolyLine

# Crear líneas entre el sitio de lanzamiento y cada punto de interés
city_line = PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (city_lat, city_lon)], 
    weight=2, 
    color='red'
)

railway_line = PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (railway_lat, railway_lon)], 
    weight=2, 
    color='blue'
)

highway_line = PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)], 
    weight=2, 
    color='green'
)

# Agregar las líneas al mapa
site_map.add_child(city_line)
site_map.add_child(railway_line)
site_map.add_child(highway_line)

site_map
